# Introduction

This is a very basic notebook to show how can you further increase your score combining the predictions of all the models you have used in this competiton. In this notebook I have used the ***best*** predictions of XGBoost, Lgbm, Catboost, logisticRegression, ridgeClassifier and some other models.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Here the model predictions on the training set will be used as the features of the dataset for this second level classification and the labels will be the same.

In [ ]:
train=pd.DataFrame({
    'xgb':pd.read_csv('/content/drive/MyDrive/ML/xgb_train.csv')['target'],
    'lr':pd.read_csv('/content/drive/MyDrive/ML/lr_train.csv')['target'],
    'lgbm':pd.read_csv('/content/drive/MyDrive/ML/lgbm_train.csv')['target'],
    'cat':pd.read_csv('/content/drive/MyDrive/ML/cat_train.csv')['target'],
    'sgd':pd.read_csv('/content/drive/MyDrive/ML/sgd_train.csv')['target'],
    'ridge':pd.read_csv('/content/drive/MyDrive/ML/ridge_train.csv')['target'],
    'lgbm_nn':pd.read_csv('/content/drive/MyDrive/ML/train_lgbm_nn.csv')['target']
})
Y=pd.read_csv('/content/drive/MyDrive/ML/train_TPSMar.csv')['target']

These are the predictions on the test set by each of the models

In [ ]:
test=pd.DataFrame({
    'xgb':pd.read_csv('/content/drive/MyDrive/ML/xgb_test.csv')['target'],
    'lr':pd.read_csv('/content/drive/MyDrive/ML/lr_test.csv')['target'],
    'lgbm':pd.read_csv('/content/drive/MyDrive/ML/lgbm_test.csv')['target'],
    'cat':pd.read_csv('/content/drive/MyDrive/ML/cat_test.csv')['target'],
    'sgd':pd.read_csv('/content/drive/MyDrive/ML/sgd_test.csv')['target'],
    'ridge':pd.read_csv('/content/drive/MyDrive/ML/ridge_test.csv')['target'],
    'lgbm_nn':pd.read_csv('/content/drive/MyDrive/ML/test_lgbm_nn.csv')['target']
})

In [ ]:
test_id=pd.read_csv('/content/drive/MyDrive/ML/test_TPSMar.csv')['id']

I have used a basic 10 fold RidgeClassifier to make predictions on the second level test set which worked out to be the best for me. I did try using basic XGB, Lgbm, RandomForest but RidgeClassifier worked out to be the best of all those.

In [ ]:
from sklearn.linear_model import RidgeClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

kf=StratifiedKFold(n_splits=10,shuffle=True,random_state=100)
pred=np.zeros((test.shape[0],))

for fold,(train_idx,valid_idx) in enumerate(kf.split(train,Y)):
    X1,X2,Y1,Y2=train.iloc[train_idx],train.iloc[valid_idx],Y.iloc[train_idx],Y.iloc[valid_idx]
    model=CalibratedClassifierCV(RidgeClassifier())
    model.fit(X1,Y1)
    score=roc_auc_score(model.predict(X2),Y2)
    print(score,' ',fold)
    pred+=model.predict_proba(test)[:,1]/10
  

In [ ]:
pd.DataFrame({
    'id':test_id,
    'target':pred
}).to_csv('stack.csv',index=False)

With stacking I have seen lot of improvement in the final score. My 10 fold Xgb best score was 0.89227 and 10 fold lgbm best score was around 0.89220. Combining these and some other basic models with a second level classification using RidgeClassifier turned out my score to be 0.89288. Try Stacking models if you see some improvement in your basic model score you will definetely score even more.

If there are any doubts with the code or something else please do comment and upvote the notebook if you found it useful.